<a href="https://colab.research.google.com/github/giramos/InformationRecovery/blob/master/Ejercicio01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Ejercicio 1** 🔥

Se prepara el entorno




In [51]:
!pip install bm25s[full]

Descargamos el fichero

In [52]:
import gdown
import zipfile
import os

# URL de descarga directa de Google Drive
url = "https://drive.google.com/uc?export=download&id=17hrYKedsi_5t59UWGN0bvYBrcUh5hHKI"

# Descargar el archivo .zip
output = "trec-covid-RI.zip"
gdown.download(url, output, quiet=False)

# Descomprimir el archivo .zip
with zipfile.ZipFile(output, 'r') as zip_ref:
    zip_ref.extractall("/content/trec-covid-RI")

print("Archivo descargado y descomprimido exitosamente.")

Downloading...
From (original): https://drive.google.com/uc?export=download&id=17hrYKedsi_5t59UWGN0bvYBrcUh5hHKI
From (redirected): https://drive.google.com/uc?export=download&id=17hrYKedsi_5t59UWGN0bvYBrcUh5hHKI&confirm=t&uuid=07c654ac-c54d-4a3c-b7f0-a3d46a137f43
To: /content/trec-covid-RI.zip
100%|██████████| 71.7M/71.7M [00:00<00:00, 163MB/s]


Archivo descargado y descomprimido exitosamente.


Se carga el archivo corpus y se prepara para poder ser indexado

In [53]:
import bm25s
import json
import Stemmer # Para estematizar términos


# Cargar la colección TREC-COVID desde el archivo corpus.jsonl
with open('/content/trec-covid-RI/corpus.jsonl', 'r', encoding='utf-8') as f:
    corpus_content = [json.loads(line) for line in f]

# Preparar el corpus para BM25S en dos formatos:
# 1. Una versión literal que mantenga la estructura original de los documentos.
# 2. Una versión de texto plano para tokenización e indexación.
corpus_verbatim = []
corpus_plaintext = []

for entry in corpus_content:
    document = {
        "id": entry["_id"],
        "title": entry["title"].lower(),
        "text": entry["text"].lower()
    }
    corpus_verbatim.append(document)
    # Incluir tanto el título como el texto en la versión de texto plano
    corpus_plaintext.append(f"{entry['title'].lower()} {entry['text'].lower()}")

# Procesar el texto aplicando un stemmer para el idioma inglés y eliminando palabras vacías durante la tokenización
stemmer = Stemmer.Stemmer("english")
corpus_tokenized = bm25s.tokenize(corpus_plaintext, stopwords="en", stemmer=stemmer, show_progress=True)

Split strings:   0%|          | 0/171332 [00:00<?, ?it/s]

Stem Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

Se crea el índice BM25

In [54]:
# Crear el retriever BM25 e indexar el corpus tokenizado
bm25_flavor = "lucene"
idf_flavor = "lucene"
retriever = bm25s.BM25(corpus=corpus_verbatim, method=bm25_flavor, idf_method=idf_flavor)
retriever.index(corpus_tokenized, show_progress=True)

# Guardar el índice en una carpeta local
index_folder = "TREC-COVID_index"
retriever.save(index_folder, corpus=corpus_verbatim)

print(f"Indexación completada y guardada como '{index_folder}'.")

DEBUG:bm25s:Building index from IDs objects


BM25S Count Tokens:   0%|          | 0/171332 [00:00<?, ?it/s]

BM25S Compute Scores:   0%|          | 0/171332 [00:00<?, ?it/s]

Finding newlines for mmindex:   0%|          | 0.00/199M [00:00<?, ?B/s]

Indexación completada y guardada como 'TREC-COVID_index'.


Guardamos el índice

In [55]:
import shutil

# Guardar el índice en una carpeta de OneDrive
onedrive_folder = "/content/drive/MyDrive/Colab Notebooks/RecuperacionInformacion/TREC-COVID_index"
if not os.path.exists(onedrive_folder):
    os.makedirs(onedrive_folder)

for file_name in os.listdir(index_folder):
    full_file_name = os.path.join(index_folder, file_name)
    if os.path.isfile(full_file_name):
        shutil.copy(full_file_name, onedrive_folder)

print(f"Indexación completada y guardada como '{index_folder}' y en OneDrive '{onedrive_folder}'.")

Indexación completada y guardada como 'TREC-COVID_index' y en OneDrive '/content/drive/MyDrive/Colab Notebooks/RecuperacionInformacion/TREC-COVID_index'.
